# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

## P2

In [ ]:
def sinplot(flip=1):
    x = np.linspace(0, 14, 100)
    for i in range(1, 7):
        plt.plot(x, np.sin(x + i * .5) * (7 - i) * flip)
sinplot()

In [ ]:
def estilo(sty = "whitegrid"):
    param = {'axes.facecolor': 'white',
             'axes.edgecolor': 'black',
             'axes.grid': True,
             'axes.axisbelow': 'line',
             'axes.labelcolor': 'black',
             'figure.facecolor': (1, 1, 1, 0),
             'grid.color': '#b0b0b0',
             'grid.linestyle': '-',
             'text.color': 'black',
             'xtick.color': 'red',
             'ytick.color': 'black',
             'xtick.direction': 'out',
             'ytick.direction': 'out',
             'lines.solid_capstyle': 'projecting',
             'patch.edgecolor': 'black',
             'image.cmap': 'viridis',
             'font.family': ['sans-serif'],
             'font.sans-serif': ['DejaVu Sans',
             'Bitstream Vera Sans',
             'Computer Modern Sans Serif',
             'Lucida Grande',
             'Verdana',
             'Geneva',
             'Lucid',
             'Arial',
             'Helvetica',
             'Avant Garde',
             'sans-serif'],
             'patch.force_edgecolor': False,
             'xtick.bottom': True,
             'xtick.top': False,
             'ytick.left': True,
             'ytick.right': False,
             'axes.spines.left': True,
             'axes.spines.bottom': True,
             'axes.spines.right': False,
             'axes.spines.top': False}
    sns.set_style(sty, param)

In [ ]:
estilo()
sinplot()

## P3

Parte 1

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin

In [3]:
class RegresionBayesianaEmpirica(BaseEstimator, RegressorMixin):

    def __init__(self, alpha_0, beta_0, tol=1e-5, maxiter=200):
        self.set_params(alpha_0=alpha_0,beta_0=beta_0,tol=tol,maxiter=maxiter)

    def get_posteriori(self, X, y, alpha, beta):
        N,d = X.shape
        S_N_1 = alpha*np.identity(d) + beta*(X.T @ X)
        S_N = np.linalg.inv(S_N_1)
        m_N = beta*((S_N @ X.T) @ y)
        return m_N, S_N

    def fit(self, X, y):
        N,d = X.shape
        alpha_old = self.alpha_0
        beta_old = self.beta_0
        print('Iteracion: 1')
        print('valores actuales, alpha:{}, beta:{}'.format(alpha_old,beta_old))
        lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
        lambdas = np.real_if_close(lambdas,tol=1)
        gamma = np.sum(lambdas/(alpha_old + lambdas))
        m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
        alpha_new = gamma/(m_N.T @ m_N)
        beta_new = (N-gamma)/(np.sum((y-m_N.T@X.T)**2))
        for k in range(2, self.maxiter+1): 
            err_alpha = abs(alpha_old - alpha_new)
            err_beta = abs(beta_old - beta_new)
            if (err_alpha < self.tol) and (err_beta < self.tol):
                break
            else:
                print('Iteracion: ', k)
                print('valores actuales, alpha:{}, beta:{}'.format(alpha_new,beta_new))
                alpha_old = alpha_new
                beta_old = beta_new
                lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
                lambdas = np.real_if_close(lambdas,tol=1)
                gamma = np.sum(lambdas/(alpha_old + lambdas))
                m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
                alpha_new = gamma/(m_N.T @ m_N)
                beta_new = (N-gamma)/(np.sum((y-m_N.T@X.T)**2))
        self.alpha_0 = alpha_new
        self.beta_0 = beta_new
        print('\nvalores finales, alpha:{}, beta:{}'.format(alpha_new,beta_new))
        m_N, S_N = self.get_posteriori(X,y,alpha_new,beta_new)
        self.m_N = m_N
        self.S_N = S_N

    def predict(self, X_, return_std=False):
        y_ = X_ @ self.m_N
        y_std = np.sqrt(1/self.beta_0 + np.diag((X_ @ self.S_N) @ X_.T))
        if return_std:
            return y_, y_std
        else:
            return y_    

Parte 2

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [5]:
'''Se cargan los datos'''
data = pd.read_csv('data.csv')
print(data.shape)
data.columns

(14118, 32)


Index(['details', 'furnished', 'metrocuadrado_index', 'n_bath', 'n_rooms',
       'price', 'surface', 'url', 'property_type', 'rent_type', 'location',
       'price_by_m2', 'n_garajes', 'tipo_producto', 'UPZ_Codigo', 'UPlTipo',
       'UPlNombre', 'UPlArea', 'nomupz', 'personas',
       'trabajoinf_ninos_5_17_anos_perc',
       'trabajoinfampliado_ninos_5_17_anos_perc',
       'jovenes_14_24_anos_nini_perc', 'indice_envegecimiento',
       'jefe_mujer_perc', 'adultos_mayores_pobres_perc', 'UPlNombre2',
       'indice_inseguridad', 'upz', 'areas_verdes_perc', 'Densidad',
       'Cluster'],
      dtype='object')

In [6]:
data.dtypes

details                                     object
furnished                                     bool
metrocuadrado_index                        float64
n_bath                                     float64
n_rooms                                    float64
price                                      float64
surface                                    float64
url                                         object
property_type                               object
rent_type                                   object
location                                    object
price_by_m2                                float64
n_garajes                                    int64
tipo_producto                              float64
UPZ_Codigo                                  object
UPlTipo                                    float64
UPlNombre                                   object
UPlArea                                    float64
nomupz                                      object
personas                       

In [7]:
'''Es necesario sacar los outliers para que funcione bien el regresor'''
for col in ['price_by_m2','surface']:
    q = data[col].quantile(0.98)
    data = data[data[col] < q]
print(data.shape)

(13553, 32)


In [8]:
'''Se definen las columnas categoricas, numericas porcentuales y numerica general'''

cat_cols = ['tipo_producto', 'rent_type', 'location','furnished','Cluster']

num_perc_cols = ['metrocuadrado_index', 'trabajoinf_ninos_5_17_anos_perc',
                 'trabajoinfampliado_ninos_5_17_anos_perc', 'jovenes_14_24_anos_nini_perc',
                 'indice_envegecimiento', 'jefe_mujer_perc', 'adultos_mayores_pobres_perc', 
                 'indice_inseguridad','areas_verdes_perc',
                 'n_rooms', 'n_bath','n_garajes']

num_cols =['Densidad']

'''Se definen la matriz de observaciones y la columna con la variable de respuesta '''
df_X = data[cat_cols + num_perc_cols + num_cols]
print(df_X.shape)
df_y = data['price_by_m2']

(13553, 18)


In [9]:
'''Se crean pipelines adecuadas'''

# categorical pipeline
cat_pipe = Pipeline([('encoding', OneHotEncoder(handle_unknown='ignore'))])

# numericals pipeline
num_perc_pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures(degree=3))])
num_pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=3))])

prep = ColumnTransformer(transformers=[('num_perc', num_perc_pipe, num_perc_cols),('num', num_pipe, num_cols), ('cat', cat_pipe, cat_cols)])

Parte 3

In [10]:
from sklearn.model_selection import train_test_split

'''Run que separa datos en train-test, hace fit a la pipeline, hace predicciones y genera scores'''
def pipe_run(X, y, pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    RMSE = np.sqrt(np.mean((y_pred - y_test)**2))
    R2 = pipeline.score(X_test, y_test)
    print('RMSE: ', RMSE)
    print('R^2: ', R2)
    return pipeline, RMSE, R2

In [11]:
'''Probar varios alpha y beta'''

alphas = np.linspace(1e-10, 1e-5, 4, endpoint=True)
betas = np.linspace(1e-10, 1e-5, 4, endpoint=True)
params = []
R2s = []

for alpha in alphas:
    for beta in betas:
        el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha,beta,tol=1e-8))])
        regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)
        params.append((alpha,beta))
        R2s.append(R2)

Iteracion: 1
valores actuales, alpha:1e-10, beta:1e-10
Iteracion:  2
valores actuales, alpha:4.63212228439486e-08, beta:2.2073495028855647e-08
Iteracion:  3
valores actuales, alpha:6.550517138924554e-08, beta:2.197033595139066e-08
Iteracion:  4
valores actuales, alpha:7.56646199706781e-08, beta:2.1906643183582256e-08

valores finales, alpha:8.001152473181834e-08, beta:2.1876601983357715e-08
RMSE:  6933.313868128578
R^2:  0.6593113272990933
Iteracion: 1
valores actuales, alpha:1e-10, beta:3.3334e-06
Iteracion:  2
valores actuales, alpha:8.617040408996218e-11, beta:2.2308133971861625e-08

valores finales, alpha:1.7799900325250775e-09, beta:2.231441822797029e-08
RMSE:  6965.649170635814
R^2:  0.6561261376140918
Iteracion: 1
valores actuales, alpha:1e-10, beta:6.666700000000001e-06
Iteracion:  2
valores actuales, alpha:6.216338926899908e-11, beta:2.2307719265833803e-08

valores finales, alpha:1.4709373670277516e-09, beta:2.231342933826537e-08
RMSE:  6967.769619298424
R^2:  0.65591674496785

In [14]:
import pickle

'''Se guarda el mejor modelo '''

alpha_opt, beta_opt = params[R2s.index(max(R2s))]
print('Modelo Optimo')
print('alpha:{}, beta:{}'.format(alpha_opt, beta_opt))
el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha_opt,beta_opt,tol=1e-8))])
regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)

# guardar modelo
filename = 'modelo.pkl'
pickle.dump(regressor, open(filename, 'wb'))
# para cargar modelo:
# loaded_model = pickle.load(open(filename, 'rb'))

Modelo Optimo
alpha:3.3334e-06, beta:1e-05
Iteracion: 1
valores actuales, alpha:3.3334e-06, beta:1e-05
Iteracion:  2
valores actuales, alpha:2.5764944758792805e-08, beta:2.2173914917241953e-08
Iteracion:  3
valores actuales, alpha:4.988176141008787e-08, beta:2.2055564433289337e-08
Iteracion:  4
valores actuales, alpha:6.763240454362764e-08, beta:2.1957652918031267e-08

valores finales, alpha:7.662353929518277e-08, beta:2.1900178907766376e-08
RMSE:  6933.061119442328
R^2:  0.6593361659394817


Parte 4

In [ ]:
'''Hay que redefinir las columnas... obtenidas de EDA'''
## JS: Falta hacer esto!

cat_cols = ['tipo_producto', 'rent_type', 'location','furnished','Cluster']

num_perc_cols = ['metrocuadrado_index', 'trabajoinf_ninos_5_17_anos_perc',
                 'trabajoinfampliado_ninos_5_17_anos_perc', 'jovenes_14_24_anos_nini_perc',
                 'indice_envegecimiento', 'jefe_mujer_perc', 'adultos_mayores_pobres_perc', 
                 'indice_inseguridad','areas_verdes_perc',
                 'n_rooms', 'n_bath','n_garajes']

num_cols =['Densidad']

'''Tambien redefinir los datos'''
df_X = data[cat_cols + num_perc_cols + num_cols]
df_y = data['price_by_m2']

'''Tambien redefinir prep, pues depende de las columnas'''
prep = ColumnTransformer(transformers=[('num_perc', num_perc_pipe, num_perc_cols),('num', num_pipe, num_cols), ('cat', cat_pipe, cat_cols)])

In [ ]:
'''Se corre y se guarda modelo, con alpha y beta optimos de parte anterior'''

el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha_opt,beta_opt,tol=1e-8))])
regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)
# guardar modelo
filename = 'modelo_EDA.pkl'
pickle.dump(regressor, open(filename, 'wb'))
# para cargar modelo:
# loaded_model = pickle.load(open(filename, 'rb'))

Parte 5

In [ ]:
from sklearn.linear_model import BayesianRidge

'''Se corre pipeline con BayesianRidge'''

ridge_pipe = Pipeline([('prep', prep), ('regressor_ridge', BayesianRidge())])
regressor_ridge, RMSE_ridge, R2_ridge = pipe_run(df_X, df_y, ridge_pipe)